In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark transformation basics").getOrCreate()
df = spark.read.csv("/Volumes/workspace/default/pyspark_sample/Employee_salary.csv", header=True,inferSchema=True)
df.show()

In [0]:
df=df.withColumnRenamed("Dept","Departments")
df=df.withColumnRenamed("EmpID","EmployeeID")
df=df.withColumn("Bonus",df.Salary*0.2)
df.show()


In [0]:
df=df.withColumn("Total_salary",df.Salary+df.Bonus)
df=df.drop("Total salary")
df.show()

In [0]:
#1.add column and cast the value
df=df.withColumn("Total_salary",df.Total_salary.cast("int"))
df=df.withColumn("Bonus",df.Bonus.cast("Double"))
df=df.withColumn("Salary",df.Salary.cast("Double"))
df=df.withColumn("Experience",df.Experience.cast("int"))
df.show()


In [0]:
#2.Filter and add column
from pyspark.sql.functions import when

df.filter(df.Salary > 45000)
df.show()
df = df.withColumn(
    "Level",
    when(df.Experience >= 5, "Lead")
    .otherwise(when(df.Experience > 3, "Senior")
    .otherwise("Junior"))
)
df.show()

In [0]:
#aggregation functions and GroupBy
df.groupBy("Departments").count().show()
df.groupBy("Departments").agg({"Salary": "sum"}).show()
df.groupby("City").agg({"Salary": "avg"}).show()
df.groupby("Departments").count().show()
df.groupby("Departments").agg({"Salary": "avg"}).show()
df.groupby("Departments").agg({"Salary": "max"}).show()
df.groupby("Departments").agg({"Salary": "min"}).show()


In [0]:
# Window function
from pyspark.sql.window import Window
#from pyspark.sql.functions import col
from pyspark.sql.functions import *
#from pyspark.sql.functions import rank
windowspec=Window.partitionBy("Departments").orderBy(col("Salary").desc())
df=df.withColumn("rank",rank().over(windowspec))
df.show()
df.filter(df.rank==1).show()


In [0]:
#join
df1 = spark.read.csv("/Volumes/workspace/default/pyspark_sample/Employee_salary.csv", header=True, inferSchema=True)
df1.show()

departments = [
    {"Dept": "HR", "location": "Block A"},
    {"Dept": "IT", "location": "Block B"},
    {"Dept": "Sales", "location": "Block C"},
    {"Dept": "Marketing", "location": "Block D"},
    {"Dept": "Finance", "location": "Block E"},
    {"Dept": "Engineering", "location": "Block F"},
]

departments_df = spark.createDataFrame(departments)
df1.join(departments_df, df1.Dept == departments_df.Dept, "inner").show()
df1.join(broadcast(departments_df), "Dept", "inner").show()


